# Using Tensorboard

In [1]:
import tensorflow as tf
a = tf.constant(10, name="a")
b = tf.constant(90, name="b")
y = tf.Variable(a + 2 * b, name="y")

model = tf.initialize_all_variables()
with tf.Session() as session:
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter\
    ("/tmp/tensorflowlogs", session.graph)
    session.run(model)
    print(session.run(y))
    
# Open terminal and run command:
# tensorboard --logdir=/tmp/tensorflowlogs

190


# MNIST Convolutional NN

In [13]:
import input_data
import numpy as np
import matplotlib.pyplot as plt

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
n_input, n_classes = 784, 10

# Hyperparameters
learning_rate  = 1e-3
training_iters = 1e5
batch_size     = 128
display_step   = 10
dropout        = 0.75 # dropout probability
keep_prob = tf.placeholder(tf.float32) # (for dropout)

x = tf.placeholder(tf.float32, [None, n_input])
_X = tf.reshape(x, shape=[-1, 28, 28, 1])  # Assuming -1 will be the number of samples?
y = tf.placeholder(tf.float32, [None, n_classes]) # output probabilities

In [4]:
def conv2d(img, w, b):
    """ 
    Args:
        img --  input tensor of shape [batchsize, in_height, in_width, in_channels]
                where channels may be, e.g. 3 for RGB color
        w   --  filter with shape [f_height, f_width, in_channels, n_feat_maps]
        b   --  bias for each feature map (number of biases = depth of the conv layer)
    """
    return tf.nn.relu(tf.nn.bias_add(\
            tf.nn.conv2d(img, w, strides=[1,1,1,1], padding='SAME'), b))

def max_pool(img, k=2):
    """
    Args:
        img -- output of a conv layer
        k   -- window size and stride (small)
    """
    return tf.nn.max_pool(img, 
                         ksize=[1, k, k, 1], 
                         strides=[1, k, k, 1], 
                         padding='SAME')

In [5]:
# __________ Weights and biases for all layers __________

# 5x5 conv, 1 input, 32 outputs
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32])) 
bc1 = tf.Variable(tf.random_normal([32]))

# 5x5 conv, 32 inputs, 64 outputs
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64])) 
bc2 = tf.Variable(tf.random_normal([64]))

# FC, 7*7*64 inputs, 1024 outputs
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))

# Output layer. 1024 inputs, 10 outputs.
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))

In [10]:
# __________ The layers __________

# [In] --> Conv --> Pool --> Dropout
conv1 = conv2d(_X, wc1, bc1)
conv1 = max_pool(conv1, k=2)
conv1 = tf.nn.dropout(conv1, keep_prob)

# --> Conv --> Pool --> Dropout
conv2 = conv2d(conv1, wc2, bc2)
conv2 = max_pool(conv2, k=2)
conv2 = tf.nn.dropout(conv2, keep_prob)

# --> Fully-Connected[ReLu] --> Dropout
# (reshape conv2 out essentially by flattening all maps into single list)
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])
dense1 = tf.nn.relu( tf.add( tf.matmul( dense1, wd1 ), bd1 ) )
dense1 = tf.nn.dropout(dense1, keep_prob)

# Output prediction.
pred = tf.add(tf.matmul(dense1, wout), bout)

## Cost and Optimizing

$$
\text{cost} = \frac{1}{n} \sum_{i = 1}^{n_{out}} y_i \log\bigg( \frac{e^{z_i}}{\sum_k e^{z_k}}\bigg)
$$

In [15]:
# ______________ Training _______
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [16]:
# _______ Evaluation _____
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy     = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [17]:
# _________ BLAST OFF _____________
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print("Iter", step * batch_size, 
                 ", Minibatch Loss={:.6f}".format(loss),
                 ", Training Accuracy={:.5f}".format(acc))
        
        step += 1
    print("Optimization finished. Am robot.")






















('Iter', 1280, ', Minibatch Loss=25021.994141', ', Training Accuracy=0.26562')
('Iter', 2560, ', Minibatch Loss=20956.230469', ', Training Accuracy=0.41406')
('Iter', 3840, ', Minibatch Loss=10467.468750', ', Training Accuracy=0.54688')
('Iter', 5120, ', Minibatch Loss=6931.669434', ', Training Accuracy=0.64844')
('Iter', 6400, ', Minibatch Loss=11381.146484', ', Training Accuracy=0.58594')
('Iter', 7680, ', Minibatch Loss=6931.756836', ', Training Accuracy=0.67188')
('Iter', 8960, ', Minibatch Loss=6043.289062', ', Training Accuracy=0.70312')
('Iter', 10240, ', Minibatch Loss=2950.967041', ', Training Accuracy=0.78906')
('Iter', 11520, ', Minibatch Loss=4387.661133', ', Training Accuracy=0.79688')
('Iter', 12800, ', Minibatch Loss=4279.759277', ', Training Accuracy=0.78125')
('Iter', 14080, ', Minibatch Loss=2511.234863', ', Training Accuracy=0.84375')
('Iter', 15360, ', Minibatch Loss=3200.528809', ', Training Accuracy=0.79688')
('Iter', 16640, ', Minibatch Loss=2861.273438', ', Trai